In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import matplotlib.cm as cm
import matplotlib.dates as mdates
from scipy.signal import find_peaks
import openmeteo_requests
import requests_cache
from datetime import date, timedelta
import seaborn as sns
import numpy as np
import os
import sys
import pygam
import sklearn
sys.path.append("..")
from pygam import LinearGAM, f, s, te, l 
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

citing pyGAM: Servén D., Brummitt C. (2018). pyGAM: Generalized Additive Models in Python. Zenodo. DOI: 10.5281/zenodo.1208723

Prepare mstl data for GAM 

In [2]:
# data load 
mstl_data = pd.read_csv("../data/mstl_results_clean.csv", low_memory=False)

# Übersicht über mstl_data Columns
print(mstl_data.columns.tolist())

['timestamp', 'city', 'counter_site', 'longitude', 'latitude', 'count', 'count_capped', 'log_count', 'trend', 'seasonal_24', 'seasonal_168', 'seasonal_8766', 'residual']


In [3]:
# extrahiere benötigte Spalten für GAMS
residual_data = mstl_data[['timestamp',
                           'city',
                           'counter_site',
                           'seasonal_8766',
                           'residual']].copy()

# filtert Zeilen raus, in denen anstatt Werten die Columnnames stehen
residual_data = residual_data[residual_data['timestamp']!='timestamp']

residual_data['id'] = residual_data.groupby(['city', 'counter_site']).ngroup()
residual_data['year_plus_res'] = residual_data['residual'].astype(float).values + residual_data['seasonal_8766'].astype(float).values
residual_data.head(2)

,timestamp,city,counter_site,seasonal_8766,residual,id,year_plus_res
0,2012-12-31 23:00:00+00:00,Landeshauptstadt Stuttgart,König-Karls-Brücke Barometer,-1.8545503347710663,0.30620344321336024,5,-1.548347
1,2013-01-01 00:00:00+00:00,Landeshauptstadt Stuttgart,König-Karls-Brücke Barometer,-0.9312113949920525,0.7172102469645241,5,-0.214001


In [4]:
# Referenzzeitpunkt (UTC)
ref = pd.Timestamp(min(residual_data['timestamp'].tolist())) 
print(ref)

# 1) Spalte vektorisert in Datetime konvertieren (einmalig)
t_0 = pd.to_datetime(residual_data['timestamp'], utc=True, errors='coerce')

# 2) Differenz in Stunden berechnen (vektorisiert)
residual_data['timestamp_number'] = (t_0 - ref).dt.total_seconds() / 3600.0
# => timestamp_number ist jetzt die Anzahl Stunden seit Referenzzeitpunkt

# 
residual_data['weekday'] = t_0.dt.dayofweek.astype(float).values            # 0–6
residual_data['hour'] = t_0.dt.hour.astype(float).values                    # 0–23

# assertion: values in weekday are in [0, 1, 2, 3, 4, 5, 6] 
assert set(residual_data['weekday']) == set(range(7))
# assertion: values in hour are in [0, 1, ..., 22, 23]
assert set(residual_data['hour']) == set(range(24))

2012-12-31 23:00:00+00:00


Prepare weather data for GAM

In [5]:
# load weather data 
weather_data = pd.read_csv("../data/weather_per_city.csv", low_memory=False)
print(weather_data.head(2))
print(weather_data.isna().sum())
print(weather_data.columns.tolist())

                        date  temperature_2m  apparent_temperature  rain  \
0  2012-12-30 23:00:00+00:00          5.8285              1.619927   0.0   
1  2012-12-31 00:00:00+00:00          5.8285              1.688656   0.0   

   snowfall  forecast_temperature_2m  forecast_apparent_temperature  \
0       0.0                      NaN                            NaN   
1       0.0                      NaN                            NaN   

   forecast_rain  forecast_snowfall                        city  
0            NaN                NaN  Landeshauptstadt Stuttgart  
1            NaN                NaN  Landeshauptstadt Stuttgart  
date                                   0
temperature_2m                         0
apparent_temperature                   0
rain                                   0
snowfall                               0
forecast_temperature_2m          1586134
forecast_apparent_temperature    1586134
forecast_rain                    1586156
forecast_snowfall              

In [25]:
cities = residual_data['city'].unique()
print(cities)

['Landeshauptstadt Stuttgart' 'Stadt Freiburg' 'Stadt Heidelberg'
 'Stadt Ludwigsburg' 'Stadt Mannheim' 'Stadt Reutlingen' 'Stadt Tübingen']


In [11]:
def split_data(X_weather: pd.DataFrame, X_forecast: pd.DataFrame, y: pd.Series, site_col: str = "id", time_col: str = "timestamp_number", train_frac: float = 0.8,):
# Splitte Daten in Trainings- und Testset basierend auf id 
    
    train_idx = []
    test_idx = []

    for site, group in X_weather.groupby("id"):
        group = group.sort_values(time_col)
        split = int(train_frac * len(group))

        train_idx.extend(group.index[:split])
        test_idx.extend(group.index[split:])
    
    # DataFrames / Series splitten
    X_weather_train = X_weather.loc[train_idx].copy()
    X_weather_test  = X_weather.loc[test_idx].copy()

    X_forecast_train = X_forecast.loc[train_idx].copy()
    X_forecast_test  = X_forecast.loc[test_idx].copy()

    y_train = y.loc[train_idx].copy()
    y_test  = y.loc[test_idx].copy()

    assert set(X_weather_train[site_col]) == set(X_weather_test[site_col])
    assert set(X_forecast_train[site_col]) == set(X_forecast_test[site_col])

    assert len(y_train) == len(X_weather_train) == len(X_forecast_train)
    assert len(y_test) == len(X_weather_test) == len(X_forecast_test)

    return X_weather_train, X_weather_test, X_forecast_train, X_forecast_test, y_train, y_test

In [19]:
# mean und std berechnen nur auf trainingsdaten und dann auf alle Daten anwenden

# Iteriere über alle Städte
# berechne je Stadt normierte Wetterdaten
# speichere für jede Stadt die entsprechenden Daten in einem Dictionary

prepared_data_dict = {}

for city in cities:

    city_data = residual_data[residual_data['city'] == city]
    w = weather_data[weather_data['city'] == city].copy()

    # merge with weather data by city and date (in weather data) and timestamp (in city_data)
    merged_data = pd.merge(city_data, w, left_on=['city', city_data['timestamp']], right_on=['city', w['date']], how='left')

    # Features und Zielvariable definieren
    feature_cols_weather = ['timestamp_number', 'id', 'temperature_2m', 'rain', 'snowfall', 'weekday', 'hour']
    feature_cols_forecast = ['timestamp_number', 'id', 'forecast_temperature_2m', 'forecast_rain', 'forecast_snowfall', 'weekday', 'hour']
    X_weather = merged_data[feature_cols_weather].astype(float)
    X_forecast = merged_data[feature_cols_forecast].astype(float)
    y = merged_data['year_plus_res'] 

    # drop rows with NaN values in X or y
    valid_idx = (y.notna() &
                X_weather.notna().all(axis=1) &
                X_forecast.notna().all(axis=1)
                )
    X_weather = X_weather.loc[valid_idx].copy()
    X_forecast = X_forecast.loc[valid_idx].copy()
    y = y.loc[valid_idx].copy()


    X_weather_train, X_weather_test, X_forecast_train, X_forecast_test, y_train, y_test = split_data(X_weather, X_forecast, y)
    
    # Normalisiere Wetterdaten
    for col in ['temperature_2m', 'rain', 'snowfall']:
        mean = X_weather_train[col].mean()
        std = X_weather_train[col].std()

        X_weather_train[col] = (X_weather_train[col] - mean) / std
        X_weather_test[col]  = (X_weather_test[col] - mean) / std

    for col in ['forecast_temperature_2m', 'forecast_rain', 'forecast_snowfall']:
        mean = X_forecast_train[col].mean()
        std = X_forecast_train[col].std()

        X_forecast_train[col] = (X_forecast_train[col] - mean) / std
        X_forecast_test[col]  = (X_forecast_test[col] - mean) / std

    # speichere Featurenames:
    feature_names_weather = X_weather_train.columns.tolist()
    feature_names_forecast = X_forecast_train.columns.tolist()

    # convert to numpy arrays
    X_weather_train = X_weather_train.values
    X_weather_test  = X_weather_test.values

    X_forecast_train = X_forecast_train.values
    X_forecast_test = X_forecast_test.values

    y_train = y_train.values
    y_test  = y_test.values

    prepared_data_dict[city] = {
        'X_weather_train': X_weather_train,
        'X_weather_test': X_weather_test,
        'X_forecast_train': X_forecast_train,
        'X_forecast_test': X_forecast_test,
        'y_train': y_train,
        'y_test': y_test,
        'feature_cols_weather': feature_names_weather,
        'feature_cols_forecast': feature_names_forecast
    
    }

    assert len(X_weather_train) == len(y_train)
    assert len(X_weather_test) == len(y_test)
    assert len(X_forecast_train) == len(y_train)
    assert len(X_forecast_test) == len(y_test)  
    

In [20]:
prepared_data_dict['Stadt Tübingen']['feature_cols_weather']

['timestamp_number',
 'id',
 'temperature_2m',
 'rain',
 'snowfall',
 'weekday',
 'hour']

# Natalies Modell
gam7 = LinearGAM(
    # Aktuelle Wetterdaten
    s(3, n_splines=4) +           # temp
    s(4, n_splines=4) +           # rain
    s(5, n_splines=4) +           # snow

    # Interaktionen mit Zeit
    te(3, 4, n_splines=4) +                             # temp × rain
    te(3, 6, n_splines=[4, 5], basis=['ps', 'cp']) +    # temp × weekday (zyklisch)
    te(4, 6, n_splines=[4, 5], basis=['ps', 'cp']) +    # rain × weekday (zyklisch)
    te(3, 7, n_splines=[4, 5], basis=['ps', 'cp']) +    # temp × hour (zyklisch)
    te(4, 7, n_splines=[4, 5], basis=['ps', 'cp']) +    # rain × hour (zyklisch)

    # unterschiede je nach ID
    s(3, by=2, n_splines=4) +   # temp nach ID
    s(4, by=2, n_splines=4)    # rain nach ID
    ).gridsearch(X_train, y_train)

In [26]:
#cities = ["Stadt Tübingen"]
metrics_dict = {}

# iteriere über alle Städte
# trainiere jeweils ein GAM auf allen Daten der Stadt, auf den tatsächlichen Wetterdaten und auf den Wettervorhersagen
# berechne Metriken und speichere sie in einem Dictionary
for city in cities:
    X_weather_train = prepared_data_dict[city]['X_weather_train']
    X_weather_test  = prepared_data_dict[city]['X_weather_test']

    X_forecast_train = prepared_data_dict[city]['X_forecast_train']
    X_forecast_test  = prepared_data_dict[city]['X_forecast_test']

    y_train = prepared_data_dict[city]['y_train']
    y_test  = prepared_data_dict[city]['y_test']


    # GAM Modell definieren
    gam_1 = LinearGAM(
        f(1) +                        # ID als Faktorvariable
        # Aktuelle Wetterdaten
        s(2, n_splines=4) +           # temp
        s(3, n_splines=4) +           # rain
        s(4, n_splines=4) +           # snow

        # Interaktionen mit Zeit
        te(2, 3, n_splines=4) +                             # temp × rain
        te(2, 5, n_splines=[4, 5], basis=['ps', 'cp']) +    # temp × weekday (zyklisch)
        te(3, 5, n_splines=[4, 5], basis=['ps', 'cp']) +    # rain × weekday (zyklisch)
        te(2, 6, n_splines=[4, 5], basis=['ps', 'cp']) +    # temp × hour (zyklisch)
        te(3, 6, n_splines=[4, 5], basis=['ps', 'cp']) +    # rain × hour (zyklisch)

        # unterschiede je nach ID
        s(2, by=1, n_splines=4) +   # temp nach ID
        s(3, by=1, n_splines=4)     # rain nach ID
        )
    gam_1.gridsearch(X_weather_train, y_train)
    

    gam_2 = LinearGAM(
        f(1) +                        # ID als Faktorvariable
        # Aktuelle Wetterdaten
        s(2, n_splines=4) +           # temp
        s(3, n_splines=4) +           # rain
        s(4, n_splines=4) +           # snow

        # Interaktionen mit Zeit
        te(2, 3, n_splines=4) +                             # temp × rain
        te(2, 5, n_splines=[4, 5], basis=['ps', 'cp']) +    # temp × weekday (zyklisch)
        te(3, 5, n_splines=[4, 5], basis=['ps', 'cp']) +    # rain × weekday (zyklisch)
        te(2, 6, n_splines=[4, 5], basis=['ps', 'cp']) +    # temp × hour (zyklisch)
        te(3, 6, n_splines=[4, 5], basis=['ps', 'cp']) +    # rain × hour (zyklisch)

        # unterschiede je nach ID
        s(2, by=1, n_splines=4) +   # temp nach ID
        s(3, by=1, n_splines=4)     # rain nach ID
        )
    gam_2.gridsearch(X_forecast_train, y_train)


    stats_1 = gam_1.statistics_
    y_hat_1 = gam_1.predict(X_weather_test)

    metrics_dict[city + ' weather'] = {
        'lambdas': gam_1.lam,
        'AIC': stats_1.get('AIC'),
        'edof': stats_1.get('edof'),
        'GCV': stats_1.get('GCV'),
        'r2': stats_1.get('pseudo_r2'),
        'rmse_train': float(np.sqrt(np.mean((y_train - gam_1.predict(X_weather_train))**2))),
        'rmse_test': float(np.sqrt(np.mean((y_test - y_hat_1) ** 2))),
    }

    stats_2 = gam_2.statistics_
    y_hat_2 = gam_2.predict(X_forecast_test)

    metrics_dict[city + ' forecast'] = {
        'lambdas': gam_2.lam,
        'AIC': stats_2.get('AIC'),
        'edof': stats_2.get('edof'),
        'GCV': stats_2.get('GCV'),
        'r2': stats_2.get('pseudo_r2'),
        'rmse_train': float(np.sqrt(np.mean((y_train - gam_2.predict(X_forecast_train))**2))),
        'rmse_test': float(np.sqrt(np.mean((y_test - y_hat_2) ** 2))),
    }

    print(f"Completed GAM training and metrics calculation for city: {city}")

  0% (0 of 11) |                         | Elapsed Time: 0:00:00 ETA:  --:--:--
  9% (1 of 11) |##                       | Elapsed Time: 0:00:13 ETA:   0:02:14
 18% (2 of 11) |####                     | Elapsed Time: 0:00:26 ETA:   0:02:00
 27% (3 of 11) |######                   | Elapsed Time: 0:00:41 ETA:   0:01:50
 36% (4 of 11) |#########                | Elapsed Time: 0:00:56 ETA:   0:01:38
 45% (5 of 11) |###########              | Elapsed Time: 0:01:10 ETA:   0:01:24
 54% (6 of 11) |#############            | Elapsed Time: 0:01:25 ETA:   0:01:10
 63% (7 of 11) |###############          | Elapsed Time: 0:01:39 ETA:   0:00:56
 72% (8 of 11) |##################       | Elapsed Time: 0:01:54 ETA:   0:00:42
 81% (9 of 11) |####################     | Elapsed Time: 0:02:08 ETA:   0:00:28
 90% (10 of 11) |#####################   | Elapsed Time: 0:02:22 ETA:   0:00:14
100% (11 of 11) |########################| Elapsed Time: 0:02:37 Time:  0:02:37
  0% (0 of 11) |                        

Completed GAM training and metrics calculation for city: Landeshauptstadt Stuttgart


  9% (1 of 11) |##                       | Elapsed Time: 0:00:02 ETA:   0:00:24
 18% (2 of 11) |####                     | Elapsed Time: 0:00:05 ETA:   0:00:23
 27% (3 of 11) |######                   | Elapsed Time: 0:00:07 ETA:   0:00:21
 36% (4 of 11) |#########                | Elapsed Time: 0:00:10 ETA:   0:00:18
 45% (5 of 11) |###########              | Elapsed Time: 0:00:13 ETA:   0:00:16
 54% (6 of 11) |#############            | Elapsed Time: 0:00:16 ETA:   0:00:13
 63% (7 of 11) |###############          | Elapsed Time: 0:00:18 ETA:   0:00:10
 72% (8 of 11) |##################       | Elapsed Time: 0:00:21 ETA:   0:00:08
 81% (9 of 11) |####################     | Elapsed Time: 0:00:24 ETA:   0:00:05
 90% (10 of 11) |#####################   | Elapsed Time: 0:00:26 ETA:   0:00:02
100% (11 of 11) |########################| Elapsed Time: 0:00:29 Time:  0:00:29
  0% (0 of 11) |                         | Elapsed Time: 0:00:00 ETA:  --:--:--
  9% (1 of 11) |##                      

Completed GAM training and metrics calculation for city: Stadt Freiburg


  9% (1 of 11) |##                       | Elapsed Time: 0:00:07 ETA:   0:01:11
 18% (2 of 11) |####                     | Elapsed Time: 0:00:15 ETA:   0:01:08
 27% (3 of 11) |######                   | Elapsed Time: 0:00:23 ETA:   0:01:01
 36% (4 of 11) |#########                | Elapsed Time: 0:00:30 ETA:   0:00:53
 45% (5 of 11) |###########              | Elapsed Time: 0:00:38 ETA:   0:00:45
 54% (6 of 11) |#############            | Elapsed Time: 0:00:45 ETA:   0:00:37
 63% (7 of 11) |###############          | Elapsed Time: 0:00:53 ETA:   0:00:30
 72% (8 of 11) |##################       | Elapsed Time: 0:01:00 ETA:   0:00:22
 81% (9 of 11) |####################     | Elapsed Time: 0:01:07 ETA:   0:00:15
 90% (10 of 11) |#####################   | Elapsed Time: 0:01:17 ETA:   0:00:07
100% (11 of 11) |########################| Elapsed Time: 0:01:25 Time:  0:01:25
  0% (0 of 11) |                         | Elapsed Time: 0:00:00 ETA:  --:--:--
  9% (1 of 11) |##                      

...

Completed GAM training and metrics calculation for city: Stadt Mannheim


  9% (1 of 11) |##                       | Elapsed Time: 0:00:07 ETA:   0:01:10
 18% (2 of 11) |####                     | Elapsed Time: 0:00:14 ETA:   0:01:04
 27% (3 of 11) |######                   | Elapsed Time: 0:00:21 ETA:   0:00:57
 36% (4 of 11) |#########                | Elapsed Time: 0:00:29 ETA:   0:00:50
 45% (5 of 11) |###########              | Elapsed Time: 0:00:36 ETA:   0:00:43
 54% (6 of 11) |#############            | Elapsed Time: 0:00:43 ETA:   0:00:36
 63% (7 of 11) |###############          | Elapsed Time: 0:00:51 ETA:   0:00:29
 72% (8 of 11) |##################       | Elapsed Time: 0:00:58 ETA:   0:00:21
 81% (9 of 11) |####################     | Elapsed Time: 0:01:05 ETA:   0:00:14
 90% (10 of 11) |#####################   | Elapsed Time: 0:01:13 ETA:   0:00:07
100% (11 of 11) |########################| Elapsed Time: 0:01:20 Time:  0:01:20
  0% (0 of 11) |                         | Elapsed Time: 0:00:00 ETA:  --:--:--
  9% (1 of 11) |##                      

Completed GAM training and metrics calculation for city: Stadt Reutlingen


  9% (1 of 11) |##                       | Elapsed Time: 0:00:01 ETA:   0:00:17
 18% (2 of 11) |####                     | Elapsed Time: 0:00:03 ETA:   0:00:16
 27% (3 of 11) |######                   | Elapsed Time: 0:00:05 ETA:   0:00:15
 36% (4 of 11) |#########                | Elapsed Time: 0:00:07 ETA:   0:00:13
 45% (5 of 11) |###########              | Elapsed Time: 0:00:09 ETA:   0:00:11
 54% (6 of 11) |#############            | Elapsed Time: 0:00:11 ETA:   0:00:09
 63% (7 of 11) |###############          | Elapsed Time: 0:00:13 ETA:   0:00:07
 72% (8 of 11) |##################       | Elapsed Time: 0:00:15 ETA:   0:00:05
 81% (9 of 11) |####################     | Elapsed Time: 0:00:17 ETA:   0:00:03
 90% (10 of 11) |#####################   | Elapsed Time: 0:00:18 ETA:   0:00:01
100% (11 of 11) |########################| Elapsed Time: 0:00:20 Time:  0:00:20
  0% (0 of 11) |                         | Elapsed Time: 0:00:00 ETA:  --:--:--
  9% (1 of 11) |##                      

Completed GAM training and metrics calculation for city: Stadt Tübingen


In [27]:
import json

with open("metrics.json", "w") as f:
    json.dump(metrics_dict, f, indent=4)

In [28]:
# print overview of metrics
for key, value in metrics_dict.items():
    print(f"{key}:")
    for metric, metric_value in value.items():
        if metric != 'lambdas':  # Skip printing lambdas for brevity
            print(f"  {metric}: {metric_value}")
    print()

Landeshauptstadt Stuttgart weather:
  AIC: 619016.0991378343
  edof: 80.42838206512829
  GCV: 0.2058059443147248
  r2: OrderedDict([('explained_deviance', np.float64(0.3729038080980184)), ('McFadden', np.float64(0.6788192809770884)), ('McFadden_adj', np.float64(0.3210042754568273))])
  rmse_train: 0.45355467746462075
  rmse_test: 0.4271326937093402

Landeshauptstadt Stuttgart forecast:
  AIC: 614139.994988745
  edof: 77.479348449612
  GCV: 0.20377732484496042
  r2: OrderedDict([('explained_deviance', np.float64(0.37907465254137107)), ('McFadden', np.float64(0.6713158311272505)), ('McFadden_adj', np.float64(0.3285147403702495))])
  rmse_train: 0.45131759261769683
  rmse_test: 0.42533139938177145

Stadt Freiburg weather:
  AIC: 64493.27189421481
  edof: 61.607679411329954
  GCV: 0.11733281104431066
  r2: OrderedDict([('explained_deviance', np.float64(0.25502222416369125)), ('McFadden', np.float64(0.6695925373285361)), ('McFadden_adj', np.float64(0.32912570757129955))])
  rmse_train: 0.34

In [29]:
# print all lambdas
for key, value in metrics_dict.items():
    print(f"{key} lambdas: {value['lambdas']}")

Landeshauptstadt Stuttgart weather lambdas: [[np.float64(0.001)], [np.float64(0.001)], [np.float64(0.001)], [np.float64(0.001)], [[np.float64(0.001)], [np.float64(0.001)]], [[np.float64(0.001)], [np.float64(0.001)]], [[np.float64(0.001)], [np.float64(0.001)]], [[np.float64(0.001)], [np.float64(0.001)]], [[np.float64(0.001)], [np.float64(0.001)]], [np.float64(0.001)], [np.float64(0.001)]]
Landeshauptstadt Stuttgart forecast lambdas: [[np.float64(0.001)], [np.float64(0.001)], [np.float64(0.001)], [np.float64(0.001)], [[np.float64(0.001)], [np.float64(0.001)]], [[np.float64(0.001)], [np.float64(0.001)]], [[np.float64(0.001)], [np.float64(0.001)]], [[np.float64(0.001)], [np.float64(0.001)]], [[np.float64(0.001)], [np.float64(0.001)]], [np.float64(0.001)], [np.float64(0.001)]]
Stadt Freiburg weather lambdas: [[np.float64(0.001)], [np.float64(0.001)], [np.float64(0.001)], [np.float64(0.001)], [[np.float64(0.001)], [np.float64(0.001)]], [[np.float64(0.001)], [np.float64(0.001)]], [[np.float64